In [10]:
import pandas as pd
import numpy as np
import os

In [54]:
usecol_ = ["Chr", "Start", "End", "Ref", "Alt", "Func.refGene", "Otherinfo10", "Otherinfo11"]
dtype_ = {"Chr":"category",
          "Start":np.int32,
          "End":np.int32,
          "Ref":"category",
          "Alt":"category",
          "Func.refGene":"category",
          "Otherinfo10":"category",
          "Otherinfo11":"string"}

In [169]:
df = pd.read_csv("anno1.fixed.chrY.genes.VE_22251.hg38_multianno.csv", usecols=usecol_, dtype=dtype_)

# add two columns DP and AF int and float
# split otherinfo11 into 4 columns (,3) and take the 3rd column (str[2]) which has DP=415
# run .str.split again to take DP=415 to 415 only
df = pd.eval("DP = df.Otherinfo11.str.split(';',3).str[2].str.split('DP=').str[1].astype('int32')", target=df)
df = pd.eval("AF = df.Otherinfo11.str.split(';',5).str[4].str.split('AF=').str[1].astype('float')", target=df)

In [159]:
# category filters works on Chr, Ref, Alt, Func.refGene, Otherinfo10
def category_filters(**kwargs):
    new_df = df
    for key, value in kwargs.items():
        if key == "FuncrefGene":
            new_df = new_df[new_df["Func.refGene"] == value]
        else:
            new_df = new_df[new_df[key] == value]
    return new_df

In [160]:
# if you want to use Func.refGene, enter 'FuncrefGene' instead 
display(category_filters(FuncrefGene="exonic", Otherinfo10="PASS"))

,Chr,Start,End,Ref,Alt,Func.refGene,Otherinfo10,Otherinfo11,DP,AF
0,chrY,2787119,2787119,C,A,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=415;VD=7;AF=0.0169...,415,0.0169
1,chrY,2787445,2787445,T,C,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=317;VD=4;AF=0.0126...,317,0.0126
15,chrY,2865200,2865200,C,A,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=173;VD=2;AF=0.0116...,173,0.0116
16,chrY,2866887,2866887,G,T,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=232;VD=4;AF=0.0172...,232,0.0172
28,chrY,2961350,2961350,G,A,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=152;VD=2;AF=0.0132...,152,0.0132
33,chrY,2978846,2978846,T,C,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=182;VD=2;AF=0.011;...,182,0.0110
34,chrY,2979004,2979004,A,T,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=148;VD=4;AF=0.027;...,148,0.0270
37,chrY,2979454,2979454,C,A,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=232;VD=10;AF=0.043...,232,0.0431
43,chrY,5100327,5100327,G,T,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=88;VD=88;AF=1;BIAS...,88,1.0000
44,chrY,5100614,5100614,T,G,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=189;VD=189;AF=1;BI...,189,1.0000


In [176]:
# filter keeps keys (DP or AF) greater than values (ie. 'DP = 15'; keeps DP vals > 15)
def num_filter(**kwargs):
    new_df = df
    for key, value in kwargs.items():
        new_df.loc[:,key].loc[lambda x: x > value]
        # alternate, slightly less efficient vectorized method line below
        # new_df.loc[:,key].where(new_df.loc[:,key] > value,inplace=True)
        
    return new_df

In [177]:
display(num_filter(DP=15))

,Chr,Start,End,Ref,Alt,Func.refGene,Otherinfo10,Otherinfo11,DP,AF
0,chrY,2787119,2787119,C,A,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=415;VD=7;AF=0.0169...,415.0,0.0169
1,chrY,2787445,2787445,T,C,exonic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=317;VD=4;AF=0.0126...,317.0,0.0126
2,chrY,2787569,2787569,A,G,exonic,p8,SAMPLE=VE_22251;TYPE=SNV;DP=229;VD=3;AF=0.0131...,229.0,0.0131
3,chrY,2787624,2787624,C,G,UTR5,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=141;VD=2;AF=0.0142...,141.0,0.0142
4,chrY,2841892,2841892,T,A,intronic,pSTD,SAMPLE=VE_22251;TYPE=SNV;DP=80;VD=2;AF=0.025;B...,80.0,0.0250
...,...,...,...,...,...,...,...,...,...,...
225,chrY,14829046,14829046,C,A,intronic,Q10,SAMPLE=VE_22251;TYPE=SNV;DP=6;VD=3;AF=0.5;BIAS...,NaN,0.5000
226,chrY,14840618,14840618,A,G,exonic,pSTD;Q10,SAMPLE=VE_22251;TYPE=SNV;DP=65;VD=2;AF=0.0308;...,65.0,0.0308
227,chrY,18555205,18555205,G,A,intronic,PASS,SAMPLE=VE_22251;TYPE=SNV;DP=3;VD=3;AF=1;BIAS=0...,NaN,1.0000
228,chrY,18555214,18555214,G,A,intronic,p8;NM5.25,SAMPLE=VE_22251;TYPE=SNV;DP=2;VD=2;AF=1;BIAS=0...,NaN,1.0000
